# Bank Predictions

# Step 3: Develop

In [1]:
# Importing general packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import sys
sys.path.insert(0, './scripts/')

from scripts.helpers import load_data, get_smotenc, get_upsample, bank_profit, get_downsample
from scripts.Preprocessing import Preprocessing
from scripts.ModelTuner import ModelTunerCV

%matplotlib inline

In [2]:
# Importing all sci-kit learn packages
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

I need set aside a portion of the data to evaluate my final model on. This will be saved in a file called *bank_holdout.csv* in the *holdout_data* directory.

In [3]:
bank_data_full = load_data('./cleaned_data/', 'bank-full.csv', verbose=True, index=0)
bank_data_full.drop_duplicates(inplace=True)
bank_data_tuning, bank_data_holdout = train_test_split(bank_data_full, random_state=849)

--------------------------------bank-full shape---------------------------------
                                  (36013, 17)                                   


----------------------------bank-full's column types----------------------------
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object


---------------------------bank-full first five rows----------------------------
     job  marital    education default  housing    loan       contact month  \
0  other  married     basic.4y      no  not yes      no  not cellular   may   
1  other  married  high.school  not no  not yes      no  n

In [4]:
%%bash
if [ -d "holdout_data" ]; then rm -R "holdout_data"; fi
mkdir "holdout_data"

In [5]:
bank_data_holdout.to_csv('./holdout_data/bank_holdout.csv')

In [6]:
bank_data_tuning.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
10696,other,divorced,high.school,no,yes,no,not cellular,jun,tue,-1,nonexistent,1.4,94.465,-41.8,4.961,5228.1,0
3876,admin.,divorced,university.degree,not no,not yes,no,not cellular,may,fri,-1,nonexistent,1.1,93.994,-36.4,4.859,5191.0,0
35214,other,married,high.school,not no,yes,no,cellular,may,fri,-1,nonexistent,-1.8,92.893,-46.2,1.250,5099.1,1
5153,other,divorced,university.degree,no,yes,no,not cellular,may,fri,-1,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
18284,admin.,married,university.degree,no,yes,no,not cellular,jul,wed,-1,nonexistent,1.4,93.918,-42.7,4.963,5228.1,1


In [7]:
clients_train, clients_tune, subscribed_train, subscribed_tune = train_test_split(
    bank_data_tuning.drop('y', axis=1), bank_data_tuning['y'], random_state=536)

I need to make the scoring object to be passed into my hyperparameter search and cross validation score calculations. Sci-kit Learn makes a convenient function if a user wants to make their own scorer, and it is exactly what I need to make my profit function into a viable scorer.

In [8]:
profit_score = make_scorer(bank_profit, needs_proba=True)

## Engineering Features
* Ensure data is ready for modeling
* Create any new features to enhance the model

I've made a few changes to my Preprocessing class to make it more robust and utilize more of sci-kit learn's methods for feature transformations. These changes have made my code more robust and easier to read overall. 

In [9]:
# p_ will stand for processed data
p = Preprocessing(classification=True)

# Preprocessing the normal data
p_clients_train, p_subscribed_train = p.fit_transform(
    clients_train, y=subscribed_train)
p_clients_tune, p_subscribed_tune = p.transform(
    clients_tune, subscribed_tune)
p_clients_cv, p_subscribed_cv = p.fit_transform(bank_data_tuning.drop('y', axis=1), bank_data_tuning['y'])

p_clients_train.head()

,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,marital_divorced,marital_married,marital_single,housing_not yes,...,poutcome_nonexistent,poutcome_not nonexistent,month_aug,month_jul,month_jun,month_may,month_nov,month_other,default_no,default_not no
38864,-0.444991,-1.817641,-1.355619,2.048263,-1.315679,-1.565384,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
16187,-0.444991,1.011439,0.664439,-0.412122,0.965196,0.977829,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6401,-0.444991,0.834621,0.785420,0.818071,0.909343,0.529808,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
36721,-0.444991,-1.522945,-0.855778,-0.041111,-1.018150,-0.856520,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
25541,-0.444991,0.127351,-0.478510,-0.275434,0.513533,0.587773,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Creating Models
* Creating and tuning models ([Logistic Regression](#Creating-Models,-Logistic-Regression), [Gradient Boosting](#Creating-Models,-Gradient-Boosting), [Random Forest](#Creating-Models,-Random-Forest))

### Creating Models, Baseline

Before anything, I want to have a baseline of how well my classifiers will end up being. To do so, I will be creating a plain LogisticRegression model without changing any of the hyperparameters. This is just a quick and dirty way to create a classifier that might be used in an environment unfamiliar with Data Science.

In [10]:
lr = LogisticRegression()
cross_val_score(lr, p_clients_cv, p_subscribed_cv, scoring=profit_score, cv=5).mean()

/home/kyle/DSDJ/Module4/bank-predictions/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kyle/DSDJ/Module4/bank-predictions/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kyle/DSDJ/Module4/bank-predictions/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kyle/DSDJ/Module4/bank-predictions/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kyle/DSDJ/Module4/bank-predictions/venv/lib/py

0.7412951620895286

A quick note, for this project, I will be utilizing RandomizedSearchCV instead of tuning the models by hand. I want to eventually automate the model tuning process, and this method seemed to be better than the more robust, but incredibly more time consuming, GridSearchCV.

### Creating Models, Logistic Regression

In [11]:
params = {'C': sp.uniform(0.001, 5),
          'solver': ['newton-cg', 'sag', 'lbfgs']
          }
lr_tuner = ModelTunerCV(LogisticRegression(), profit_score, cv=5)

lr_tuner.tune(X=p_clients_cv, 
              y=p_subscribed_cv, 
              param_grid=params, 
              method='random')
print(f"Best Params: {lr_tuner.best_params_}")
print(f"Best Profit Score: {lr_tuner.best_score_}")

Best Params: {'C': 3.1586050615542045, 'solver': 'newton-cg'}
Best Profit Score: 0.7424693154902402


### Creating Models, Gradient Boosting

In [12]:
gbc = GradientBoostingClassifier(random_state=985)
cross_val_score(gbc, p_clients_cv, p_subscribed_cv, scoring=profit_score, cv=3).mean()

0.7427295220653614

In [ ]:
gbc_param_dist = {'learning_rate': sp.uniform(.0001, .2), # Picks from a range of [.0001, .2001]
    'subsample': sp.uniform(0.5, .5), # Picks from a range of [.5, 1]
    'min_samples_leaf': sp.randint(1, 5),
    'max_depth': sp.randint(2, 15),
    'max_features': sp.uniform(.01, .99)
}

gbc_tuner = ModelTunerCV(gbc, profit_score, cv=3)
gbc_tuner.tune(X=p_clients_cv, 
              y=p_subscribed_cv, 
              param_grid=gbc_param_dist, 
              method='random',
              n_iter=15)
print(f"Best Params: {gbc_tuner.best_params_}")
print(f"Best Profit Score: {gbc_tuner.best_score_}")

In [ ]:
gbc = GradientBoostingClassifier(random_state=985, learning_rate=.11, subsample=.8, max_depth=5, min_samples_leaf=8)
cross_val_score(gbc, p_clients_cv, p_subscribed_cv, scoring=profit_score, cv=3).mean()

To test how well the randomized cv can select good enough parameters, I manually tuned the hyperparameters for Gradient 
Boosting as well as the randomized cv. The randomized cv took about 3 minutes and tuning manually took me a few hours of mainly waiting and working on other things like updating documentation while waiting for the cross validation scores to be calculated (This may be less significant of a time sink on your machine, but mine is rather old and slow with these amount of calculations). At the end of the day, there was barely any difference in the profit scores between the hand tuned and randomized search. The time gain of the randomized search is definitely worth the potential drop in score due to uncertainty in tuning. At the very least, randomized search will almost certainly always be "good enough" for production. The performance of the hyperparameter tuning might also be due to the choice of model. I'll repeat with Random Forest which will depend heavily on its hyperparameters.

### Creating Models, Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=4215, n_estimators=100)
cross_val_score(rf, p_clients_cv, p_subscribed_cv, scoring=profit_score, cv=3).mean()

In [ ]:
rf_param_dist = {'max_depth': sp.randint(7, 20),
                 'min_samples_leaf': sp.uniform(.01, .49),
                 'max_features': sp.uniform(.5, .5)
}

rf_tuner = ModelTunerCV(rf, profit_score, cv=5)
rf_tuner.tune(X=p_clients_cv, 
              y=p_subscribed_cv, 
              param_grid=rf_param_dist, 
              method='random',
             n_iter=15)
print(f"Best Params: {rf_tuner.best_params_}")
print(f"Best Profit Score: {rf_tuner.best_score_}")

## Testing Models
* Doing a 5-fold cross validation on models

In [ ]:
def perform_cv5(model, params, X, y):
    model.set_params(**params)
    return cross_val_score(model, X, y, scoring=profit_score, cv=5).mean()

In [ ]:
print(f"Logistic Regression: {perform_cv5(lr, lr_tuner.best_params_, p_clients_cv, p_subscribed_cv)}")
print(f"Gradient Boosting: {perform_cv5(gbc, gbc_tuner.best_params_, p_clients_cv, p_subscribed_cv)}")
print(f"Random Forest: {perform_cv5(rf, rf_tuner.best_params_, p_clients_cv, p_subscribed_cv)}")

## Selecting the Best Model
* Selecting the model with the highest score for production

By an incredibly small margin, Gradient Boosting has the best profit score after averaging the 5-fold cross validation scores. The parameters will be listed below to be used in the model in the final steps.

In [ ]:
gbc = GradientBoostingClassifier(random_state=985).set_params(**gbc_tuner.best_params_)
gbc